## Problem 46 - Goldbachs other conjecture

<p>It was proposed by Christian Goldbach that every odd composite number can be written as the sum of a prime and twice a square.</p>
\begin{align}
9 = 7 + 2 \times 1^2\\
15 = 7 + 2 \times 2^2\\
21 = 3 + 2 \times 3^2\\
25 = 7 + 2 \times 3^2\\
27 = 19 + 2 \times 2^2\\
33 = 31 + 2 \times 1^2
\end{align}
<p>It turns out that the conjecture was false.</p>
<p>What is the smallest odd composite that cannot be written as the sum of a prime and twice a square?</p>


In [1]:
#first I need to find a way to check if a number is a perfect square I think. 
import numpy as np
def is_perfect_square(x: int) -> bool:
    """if the integer part of a square root squares to be equal to the original number I`m confident we have a perfect square"""
    sqx = int(np.sqrt(x))
    if sqx**2 == x:
        return True
    return False

In [2]:
from sympy import isprime

In [3]:
def satisfies_goldbach(x):
    if x%2 == 0 or isprime(x):
        return False
    else:
        for i in range(1,x):
            comp = x - i
            if isprime(comp):
                if i%2 ==0 and is_perfect_square(i//2):
                    return True
        return False
        

In [4]:
goldbach_is_satisfied = True
n = 33
while goldbach_is_satisfied:
    n+=2
    if not isprime(n):
        goldbach_is_satisfied = satisfies_goldbach(n)
print(f"{n} cannot be written as a prime and twice a square")

5777 cannot be written as a prime and twice a square


## Problem 47 - Distinct Prime Factors

<p>The first two consecutive numbers to have two distinct prime factors are:</p>
\begin{align}
14 = 2 \times 7\\
15 = 3 \times 5.
\end{align}
<p>The first three consecutive numbers to have three distinct prime factors are:</p>
\begin{align}
644 = 2^2 \times 7 \times 23\\
645 = 3 \times 5 \times 43\\
646 = 2 \times 17 \times 19.
\end{align}
<p>Find the first four consecutive integers to have four distinct prime factors each. What is the first of these numbers?</p>


In [5]:
def prime_factors(n):
    factors = []
    denom = 2
    while n!=1:
        if n%denom==0:
            n//=denom
            factors.append(denom)
        else:
            denom+=1
    return factors
    

In [6]:
a = {1,2,3}
b = {5,6,7}

len(a.intersection(b))

0

In [7]:
def check_four_consecutive_ints(n):
    factors = set(prime_factors(n))
    for i in range(1, 5):
        # print(n, i)
        new_factors = set(prime_factors(n+i))
        if len(new_factors.intersection(factors)) == 0:
            # print(factors, new_factors)
            factors = factors.union(new_factors)
        else:
            return False
    return True, n, factors
            
    

In [11]:
check_four_consecutive_ints(i)

False

In [12]:
set(prime_factors(i))

{2, 3, 41, 163}

In [13]:
set(prime_factors(i+1))

{40099}

In [15]:
set(prime_factors(i+2))

{2, 5, 401}

In [9]:
found_start_point = False
i = 1
while not found_start_point:
    found_start_point = check_four_consecutive_ints(i)
    # print(found_start_point)
    i+=1
    

KeyboardInterrupt: 

## Problem 48 - Self Powers

<p>The series, $1^1 + 2^2 + 3^3 + \cdots + 10^{10} = 10405071317$.</p>
<p>Find the last ten digits of the series, $1^1 + 2^2 + 3^3 + \cdots + 1000^{1000}$.</p>


In [ ]:
%%time

s = 0
for x in range(1, 1001, 1):
    s+= x**x

print(f"The answer is : {str(s)[-10:]}")

## Problem 49 - Prime Permutations

<p>The arithmetic sequence, $1487, 4817, 8147$, in which each of the terms increases by $3330$, is unusual in two ways: (i) each of the three terms are prime, and, (ii) each of the $4$-digit numbers are permutations of one another.</p>
<p>There are no arithmetic sequences made up of three $1$-, $2$-, or $3$-digit primes, exhibiting this property, but there is one other $4$-digit increasing sequence.</p>
<p>What $12$-digit number do you form by concatenating the three terms in this sequence?</p>


In [ ]:
from sympy import isprime
import numpy as np
from scipy.spatial.distance import cdist

#find four-digit-primes
fdps = np.array([x for x in range(1000, 10000,1) if isprime(x)])

In [ ]:
def check_gap(fdp: int, gap: int):
    """Checks if for a four digit prime and a given gap, 
    the numbers found by adding or subtracting the gap are prime
    and also comprised of the same set of digits"""
    if set(str(fdp)) != set(str(fdp+gap)):
        return False
    
    if set(str(fdp)) != set(str(fdp-gap)):
        return False
    if not isprime(fdp+gap) or not isprime(fdp-gap):
        return False
    return True
        

In [ ]:
#for every four digit prime
for fdp in fdps:
    # find all symmetrical gaps by looking for doubles in the sets of differences
    diffs, counts = np.unique(np.abs(fdps - fdp), return_counts=True)
    idxs = np.argwhere(counts == 2).flatten()
    gaps_to_check = diffs[idxs]
    #four every gap, check if the plus or minus number are prime and the same set, of so, log the instance
    for gap in gaps_to_check:
        if check_gap(fdp, gap):
            print(f"{fdp-gap}, {fdp}, {fdp+gap} with a gap of {gap}")

Interesting that the gap here is also 3330! I did not glean that from the excercise. Knowing that would have made the solution much easier I think. 

## Problem 50 - Consecutive Prime Sum

<p>The prime $41$, can be written as the sum of six consecutive primes:</p>
$$41 = 2 + 3 + 5 + 7 + 11 + 13.$$
<p>This is the longest sum of consecutive primes that adds to a prime below one-hundred.</p>
<p>The longest sum of consecutive primes below one-thousand that adds to a prime, contains $21$ terms, and is equal to $953$.</p>
<p>Which prime, below one-million, can be written as the sum of the most consecutive primes?</p>

-----
Note: The last sentence suggests that the length of the sequence we are looking for only ever produces one prime, which simplifies our search a little bit. We now only have to look for sliding windows where the rolling sum only produces one prime result:

In [ ]:
%%time

#I`ll start with gathering all primes up to a million, even though we likely only need a few dozen at most
primes_to_million = [x for x in range(1, 1000000, 2) if isprime(x)]

#keep track of the largest sequence that produces one prime result
largest_seq = 0
#keep track ot the start index of where summing the sequence produces a prime result
start_index = 0

#for every window starting from 21:
for win in range(21, len(primes_to_million)):
    #take the sum of the windowlength over the entire array
    sums = np.sum(np.lib.stride_tricks.sliding_window_view(primes_to_million, win), axis = 1)
    #if none of the sums are under one million we have left the search space and can terminate the loop
    if np.min(sums) > 1e6:
        print(f"No sums under 1e6 left at window {win}, Stopping the search")
        break
    #check which sums are primes under one million
    ip = [isprime(x) and x<1e6 for x in sums]
    #if there is exactly one, we update the largest sequence and the starting index until we are done searching
    if sum(ip) == 1:
        largest_seq = win
        start_index = np.where(ip)[0][0]
#eventually we find the solution
print(f"The largest consecutive sequence of primes that sum to a prime under one million is  {largest_seq} long.\nIt starts at {primes_to_million[start_index]}. \nThe sum is {np.sum(primes_to_million[start_index:start_index+largest_seq])}")
  